In [53]:
import numpy as np
import random
import copy
import globals as gb
from icecream import ic
from tqdm.auto import tqdm
from dataclasses import dataclass
from tree import Tree, swap_subtrees, generate_initial_solution
from utils import mse, sort_individuals, are_compatible
from mutations import mutation
from tree_node import TreeNode

## Problem selection

In [54]:
gb.initialize_globals_for_problem(2)

POPULATION_SIZE = gb.PROBLEM_SIZE * 20
OFFSPRING_SIZE = int(POPULATION_SIZE / 4)
# MAX_ITERATIONS = POPULATION_SIZE * 50
MAX_ITERATIONS = POPULATION_SIZE *75


## Steps
- We treat a possible solution as a tree. The tree has attribute root, which is the root of the tree of class TreeNode and max_depth.
- Generate random tree
    - we need each variable at least once 
    - each variable has exactly one coefficient chosen as a random float number in the range [?, ?]
    - each variable has exactly one unary operator
    - unary operator is chosen as: 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among all other unary operators
        - check if the unary operator is appliable to the variable ->
            ```
            leaves_map = {}
            for e in leaves:
                available_unary_operators = [op for op in list(UNARY_OPERATORS.keys()) if op.is_applicable(e)]
                chosen_unary_operator = 50% chance of "" (i.e. no change to the variable), 50% chance of choosing among [available_unary_operators]
                leaves_map[e] = [chosen_unary_operator]
            # leaves = [-2, 3]
            # leaves_map = {-2: square, 3: log}
            for e in leaves:
                node = leaves_map[e]
                node.left = null
                node.right = e
                # insert node to tree
            ```
    - number of leaves = nearest power of two greater than keys.length()
    - number of actual leaves = [number of leaves] * 2
    - number of coefficients = [number of leaves] - keys.length()
    - number of binrary operators = total number of nodes in  tree with [number of leaves] leaves - [number of leaves]]
    - validate tree
    - if valid, return tree
    - else, ?
- Example:
    - x.length() = 3
    - number of leaves = 4
    - number of actual leaves = 8
    - number of coefficients = 1
    - number of operands = 3

    ```bash
                    +
            /                  \
            *                    +
        /      \           /        \
      u        1          1          u
    /   \    /   \      /   \       /  \
    nul  *  nul   *    nul    *     nul *
    ```
### EA approach
- Individual is rapresented as a tree and a fitness
    - fitness is a tuple of 2 values: (-mse, right_sign_100)
        - right_sign_100 is the percentage of correct sign predictions
        - mse is the mean squared error
- Classic Genetic Programming
    - Key elements 
    - Representation: tree structures
    - Recombination: exchange of subtrees
    - Mutation: random change in trees
        - subtree mutation -> replace  entire subtree
        - point  mutation -> change single node
        - permutation -> exchange node right with left
        - hoist -> take subtree and make it root
        - expansion -> take random leaf and replace it with a new subtree
        - collapse -> take a subtree and replace it with leaf
    - Population model: generational
    - Parent selection: fitness proportional
    - Survivor selection: deterministic

##next

    - finish implementing mutations
    - find way to reduce tree dimension
    - add weights to opertors
    - add check if fitness doesn't improve , stop early
    - 

##problems
    - overflow
    - initial tree with 4xnodes has invalid values
    - 


## Individual definition

In [55]:
@dataclass
class Individual:
    genome: Tree
    fitness: tuple

## Fitness evaluation

In [56]:
def fitness(sol: Tree):
    y_computed = sol.root.evaluate_tree_from_node()
    right_sign_100 = 100 * np.sum(np.sign(y_computed) == np.sign(gb.Y)) / len(gb.Y)
    return  -mse(y_computed, gb.Y) * (np.exp((sol.get_max_depth() / 5))), right_sign_100

In [57]:
# result = np.sin((gb.X[0] * 0.499999) + ((gb.X[1] + gb.X[2]) * 0.2500002)) * 7.643017e6
# print(mse(result,gb.Y))


## EA helper functions

In [58]:
def parent_selection(population: list[Individual], scores, worst_score):
    # windowing    
    scores_prime = [(s-worst_score) for s in scores]
    # print(len(population), len(scores))
    total = sum(scores_prime)
    # if all population doesn't have some score
    if total != 0:
        probabilities = [s/total for s in scores_prime]

        parents = random.choices(population, probabilities, k=2)
            
        return parents[0], parents[1]
    
    return population[0], population[1]

def xover(parent1, parent2)-> tuple[Tree, tuple]:
    # parents = random.choices(population, k=2)

    # reproduce
    c1 = copy.deepcopy(parent1.genome)
    c2 = copy.deepcopy(parent2.genome)

    success = swap_subtrees(c1, c2)

    c_fitness = fitness(c2)

    if not success:
        # mutation
        mutation(c2)

    return c2, c_fitness

def tournament(population):
    # generate a certain number of tournaments
    n_tour = 3*gb.PROBLEM_SIZE
    winners = []
    # for each tournament find a winner for a portion of the population
    for i in range(n_tour):
        start =int( i*len(population)/n_tour)
        end =int( (i+1)*len(population)/ n_tour)
        p_i = population[start : end]
        print(p_i)
        w_i = ea(p_i, 50)
        winners.append(w_i)
    # print(f"winners: {winners}")
    winners , scores = sort_individuals(winners)
   
    # get the best winner/s
    # check if there are pari merito
    # best_score = np.max(scores)
    # return [winners[i] for i in range(len(winners)) if scores[i]==best_score]
    return winners

def fine_tuning(best: Individual):
    genome = best.genome
    best_fitness = best.fitness

    n_no_inc = 0

    leaves = genome.get_leaves_nodes()

    max_iterations = MAX_ITERATIONS // 10
    for i in range(max_iterations):
        leaf = np.random.choice(leaves)
        new_f = tune_constant(leaf,genome, best_fitness)

        if new_f > best_fitness:
            best_fitness = new_f
            n_no_inc = 0
        else:
            n_no_inc += 1

        if i % 5000 == 0 and i != 0:
            print(best_fitness)

        if n_no_inc > 1000:
            print("Early stopping")
            break

    # print("New best fitness:", best_fitness)
    # genome.draw_tree()
    best.fitness = best_fitness


def tune_constant(node: TreeNode, curr_genome, curr_fitness):
    mutation_factor = 1.002
    improve = True
    direction_changed = False
    while improve:
        if node.value in gb.VARIABLES_MAP:
            node.coefficient *= mutation_factor  # Modify costant
            if round(node.coefficient)!=0: 
                node.coefficient = round(node.coefficient,4)
        else:
            node.value *= mutation_factor
            if round(node.value)!=0: 
                node.value = round(node.value,4)
                
        new_mse, right_sign = fitness(curr_genome)
        new_fitness = ( new_mse, right_sign)
        # Aggiorna la soluzione migliore
        if new_fitness > curr_fitness:
            curr_fitness = new_fitness
            mutation_factor/=mutation_factor
        else:
            if node.value in gb.VARIABLES_MAP:
                node.coefficient /= mutation_factor  # Revert change
            else:
                node.value /= mutation_factor
            if not direction_changed:
                direction_changed = True
                mutation_factor = 0.998 # try to explore backward
            else: 
                improve = False
    
    return curr_fitness    


def ea(population, iterations=200, best=None, best_score=-1, prev_score=0):
    # print(population)
    scores = []
    offsprings = []
    worst_score = 0


    # Initialize stopping condition for maximization problem
    min_iterations = POPULATION_SIZE  # Wait for at least one generation
    patience = int(MAX_ITERATIONS / 5)  # Wait for half a generation worth of iterations
    stagnation_counter = 0
  


    for ind in population:
        ind.fitness = fitness(ind.genome)
    population, scores = sort_individuals(population)

    for i in tqdm(range(iterations)):
    
        for _ in range(OFFSPRING_SIZE):
            parent1, parent2 = parent_selection(population, scores, worst_score)
            # crossover
            if random.random() < 0.65:
                child, c_fitness = xover(parent1, parent2)
        
                offsprings.append(Individual(child, c_fitness))

            # mutation
            else :
                # take a random individual and mutate it
                ind = copy.deepcopy(random.choice([parent1,parent2]))

                mutation(ind.genome)
                # recompute fitness
                ind.fitness = fitness(ind.genome)
                offsprings.append(ind)

        population.extend(offsprings)
        offsprings = []
        
        # sort population according to score based on fitness         
        population, scores = sort_individuals(population)
        
        score = scores[0] # best sscore in currrent population

        
        if i % patience and i > min_iterations == 0:
            print(f"iteration {i} mse {mse(population[0].genome.root.evaluate_tree_from_node(), gb.Y)}")
            if score > best_score*(1.01):
                #print(f"new best score : {score}, old score : {best_score}")
                best = population[0]
                best_score = score
                stagnation_counter = 0
            elif score <= best_score*1.01 and score >= best_score*0.99:
                stagnation_counter+=1
                if stagnation_counter == 3:
                    print("STOP")
                    break
            # put again the best one in population
            else: 
                #print("valhalla")
                population.append(best)
                population,scores = sort_individuals(population)
                # print(f"valhallla {i}: scores {scores}\n")ù
                stagnation_counter = 0

        # remove worst individual
        population = population[:POPULATION_SIZE]
        scores = scores[:POPULATION_SIZE]

        worst_score = scores[OFFSPRING_SIZE-1] # if worst_score > scores[-1] else worst_score


    print(f"winner's mse {mse(population[0].genome.root.evaluate_tree_from_node(), gb.Y)}")
    
    return population[0]

## Main

In [ ]:
state = random.getstate()
population = [Individual(generate_initial_solution(seed=(i+2) * 10), 0) for i in range(POPULATION_SIZE)]
offsprings = []
random.setstate(state)

# torunament -> use initial population
winners = tournament(population)

#generate population from winner/s
population = []
for w in winners:
    for i in range(int(POPULATION_SIZE/len(winners))-1):
        ind = copy.deepcopy(w)

        mutation(ind.genome)
        # recompute fitness
        ind.fitness = fitness(ind.genome)
        population.append(ind)
    population.append(w)


population, scores = sort_individuals(population)
population[0].genome.draw_tree()

print(f"Best individual mse {mse(population[0].genome.root.evaluate_tree_from_node(), gb.Y)}")
# ea
result = ea(population, MAX_ITERATIONS, population[0], scores[0])
print("Best individual has formula:")
result.genome.print_tree()
print(f"Best individual mse {mse(result.genome.root.evaluate_tree_from_node(), gb.Y)}")
result.genome.draw_tree() 
print("tuning...")
# fine tuning of constants
fine_tuning(result)

print("Best individual has formula:")
print(f"Best individual mse {mse(result.genome.root.evaluate_tree_from_node(), gb.Y)}")
print("Best individual has formula:")
result.genome.print_tree()
result.genome.draw_tree() 


[Individual(genome=<tree.Tree object at 0x000001C30875DE50>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30854FD70>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30875F080>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30875FE90>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3087D5DC0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30869ECC0>, fitness=0)]


  0%|          | 0/200 [00:00<?, ?it/s]

winner's mse 2961516453106746.0
[Individual(genome=<tree.Tree object at 0x000001C30869E900>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30869F1A0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30869D280>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3087D6180>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30869F110>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C30869D340>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308654F80>, fitness=0)]


  0%|          | 0/200 [00:00<?, ?it/s]

winner's mse 1979610202384494.2
[Individual(genome=<tree.Tree object at 0x000001C30869E150>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308656900>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308656000>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308654860>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3086C2570>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3086C2300>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308542060>, fitness=0)]


...

  0%|          | 0/200 [00:00<?, ?it/s]

winner's mse 2129234242426896.0
[Individual(genome=<tree.Tree object at 0x000001C30878C350>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3085DF650>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308731940>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3086E80B0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3085BAAE0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3086EA840>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3086724E0>, fitness=0)]


  0%|          | 0/200 [00:00<?, ?it/s]

winner's mse 2961508522918920.0
[Individual(genome=<tree.Tree object at 0x000001C308732ED0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3085BBD70>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308540CE0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3085E1880>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3086C2CC0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C308609CD0>, fitness=0), Individual(genome=<tree.Tree object at 0x000001C3085E1EB0>, fitness=0)]


  0%|          | 0/200 [00:00<?, ?it/s]

In [47]:

print(gb.Y[0]-12065267.11*gb.X[0][0] * 0.9)

-34214523.49311387


In [48]:
x = 10e-12
y = 1e-12
z = x*y
np.tan(z)

np.float64(1e-23)

In [49]:
x = [1e-12]
y = []
print(are_compatible("+", x, 2))

True


In [50]:
x0 = gb.X[0]
x1 = gb.X[1]
res = ((x0 + -1.4220549) * (x1 / (1.1754677 / x1))) * ((x0 * ((x1 - 2.3935468) * -3.7704857e-12)) * x0)
mse(res,gb.Y)

np.float64(2562.1146162335267)

## version 2

In [51]:
""" POPULATION_SIZE = PROBLEM_SIZE*20
OFFSPRING_SIZE = POPULATION_SIZE*3
MAX_ITERATIONS = 1000 """

' POPULATION_SIZE = PROBLEM_SIZE*20\nOFFSPRING_SIZE = POPULATION_SIZE*3\nMAX_ITERATIONS = 1000 '

In [52]:
""" state = random.getstate()
initial_population = [Individual(generate_initial_solution(seed=(i+2) * 10), 0) for i in range(POPULATION_SIZE)]
offsprings = []
random.setstate(state)

# set the fitness
for ind in initial_population:
    ind.fitness = fitness(ind.genome)
initial_population, scores = sort_individuals(initial_population)
initial_population[0].genome.draw_tree()

# print(initial_population)

best = None
best_score = 0
worst_score = 0
scores = [compute_score(initial_population[i], initial_population) for i in range(POPULATION_SIZE)]

for i in tqdm(range(MAX_ITERATIONS)):
# for i in range(MAX_ITERATIONS):
    for _ in range(OFFSPRING_SIZE):
        parent1, parent2 = parent_selection(initial_population, scores, worst_score)
        # crossover
        if random.random() < 0.65:
            child, c_fitness = xover(parent1, parent2)
    
            offsprings.append(Individual(child, c_fitness))

        # mutation
        else :
            # take a random individual and mutate it
            ind = copy.deepcopy(random.choice([parent1,parent2]))

            mutation(ind.genome)
            # recompute fitness
            ind.fitness = fitness(ind.genome)
            offsprings.append(ind)

    
    # sort population according to score based on fitness         
    initial_population, scores = sort_individuals(offsprings)
    # print(f"{i}: scores {scores}\n")

    if i % 100 == 0:
        print(f"iteration {i} best individual : {initial_population[0].fitness}")
        # compute score 
        score = scores[0]
        if score > best_score:
            print(f"new best score : {score}, old score : {best_score}")
            best = initial_population[0]
            best_score = score

        # put again the best one in population
        else: 
            print("valhalla")
            initial_population.append(best)
            initial_population,scores = sort_individuals(initial_population)
            print(f"valhallla {i}: scores {scores}\n")

        worst_score = scores[OFFSPRING_SIZE-1] # if worst_score > scores[-1] else worst_score
   
   
    # remove worst individual
    initial_population = initial_population[:POPULATION_SIZE]
    scores = scores[:POPULATION_SIZE]
    # print(f"cut {i}: scores {scores}\n")
 
    # update the worst score
    # print(f"iteration {i} ")
    offsprings = []

print("Best individual has formula:")
print(f"Best individual mse {mse(initial_population[0].genome.root.evaluate_tree_from_node(VARIABLES_MAP, BINARY_OPERATORS, UNARY_OPERATORS), y)}")
initial_population[0].genome.draw_tree() """

' state = random.getstate()\ninitial_population = [Individual(generate_initial_solution(seed=(i+2) * 10), 0) for i in range(POPULATION_SIZE)]\noffsprings = []\nrandom.setstate(state)\n\n# set the fitness\nfor ind in initial_population:\n    ind.fitness = fitness(ind.genome)\ninitial_population, scores = sort_individuals(initial_population)\ninitial_population[0].genome.draw_tree()\n\n# print(initial_population)\n\nbest = None\nbest_score = 0\nworst_score = 0\nscores = [compute_score(initial_population[i], initial_population) for i in range(POPULATION_SIZE)]\n\nfor i in tqdm(range(MAX_ITERATIONS)):\n# for i in range(MAX_ITERATIONS):\n    for _ in range(OFFSPRING_SIZE):\n        parent1, parent2 = parent_selection(initial_population, scores, worst_score)\n        # crossover\n        if random.random() < 0.65:\n            child, c_fitness = xover(parent1, parent2)\n    \n            offsprings.append(Individual(child, c_fitness))\n\n        # mutation\n        else :\n            # take